### Oppgave 1: Finne landene med høyeste og laveste befolkningstetthet
Hint: Se eksemplet i veilederen

Laster inn bibliotek:

In [ ]:
from pyspark.sql import DataFrame
from pyspark.sql.types import *
import pyspark.sql.functions as F

1.1  Finne og leses inn datasett

In [ ]:
#skriv koden for å finne og lese inn datasettene om innbyggerantall 2020 og areal
spark.read.path("/felles/veiledning/pyspark/eksempler*").show(10, False)

df_areal = spark.read.path("/felles/veiledning/pyspark/eksempler/areal")
df_innbyggerantall_2020 = spark.read.path("/felles/veiledning/pyspark/eksempler/innbyggerantall/2020")
print("*********************************** AREAL ***********************************")
df_areal.show(50,False)
print("*********************************** INNBYGGERANTALL 2020 ***********************************")
df_innbyggerantall_2020.show(50,False)

1.2 Fjerne dubletter

In [ ]:
# Skriv kode for å sjekke og evt. fjerne dubletter

#Sorterer etter land og innbyggerantall og lister ut den sorterte dataframen
print("*********************************** INNBYGGERANTALL SORTERT ETTER LAND OG INNBYGGERANTALL ***********************************")

df_innbyggerantall_2020_sort = df_innbyggerantall_2020.orderBy(['Land','Innbyggerantall'], ascending=[1, 0])
df_innbyggerantall_2020_sort.show(50,False)

#Tar ut oversikt over de radene med likt "land" som forekommer flere enn en gang i dataframen, og sånn sett utgjør den samlede mengden av duplikater
df_land_dub_alle  = df_innbyggerantall_2020_sort.join(df_innbyggerantall_2020_sort.groupBy('Land')\
          .count().where('count = 1').drop('count'),
          on=['Land'],
          how='left_anti')
print("*********************************** OVERSIKT OVER DUPLIKATER ***********************************")
df_land_dub_alle.show()  

print("*********************************** INNBYGGERANTALL UTEN DUBLIKATER. HØYESTE INNBYGGERANTALL ER BEHOLDT ***********************************")
df_innbyggerantall_udub= df_innbyggerantall_2020_sort.drop_duplicates(subset=['Land'])
df_innbyggerantall_udub.show(50,False)


In [ ]:
# Skriv kode for å sjekke og evt. fjerne dubletter

#Sorterer etter land og areal og lister ut den sorterte dataframen
print("*********************************** AREALDATASETT SORTERT ETTER LAND OG AREAL ***********************************")

df_areal_sort = df_areal.orderBy(['Land','Areal'], ascending=[1, 1])
df_areal_sort.show(50,False)


#Tar ut oversikt over de radene med likt "land" som forekommer flere enn en gang i dataframen, og sånn sett utgjør den samlede mengden av duplikater
df_areal_dub_alle  = df_areal_sort.join(df_areal_sort.groupBy('Land')\
          .count().where('count = 1').drop('count'),
          on=['Land'],
          how='left_anti')
print("*********************************** OVERSIKT OVER DUPLIKATER ***********************************")
df_areal_dub_alle.show()  

print("*********************************** AREALDATSETTET UTEN DUBLETTER: MINSTE AREAL BEHOLDT ***********************************")
df_areal_udub= df_areal_sort.drop_duplicates(subset=['Land'])
df_areal_udub.show(50,False)


1.3 Koble datasett

In [ ]:
# Skriv kode for å koble sammen datasettene (innbyggerantall og areal)
df_areal_innbyggerantall = df_areal_udub.join(df_innbyggere_udub, 'Land', 'inner')
print("*********************************** SAMMENKOBLET DATASETT ***********************************")
df_areal_innbyggerantall.show(50,False)

1.4 Lage en ny variabel

In [ ]:
# Skriv kode for å lage en ny variabel for befolkningstetthet
df_areal_innbyggerantall = df_areal_innbyggerantall.withColumn('Befolkningstetthet',df_areal_innbyggerantall['Innbyggerantall']/df_areal_innbyggerantall['Areal'])
print("*********************************** SAMMENKOBLET DATASETT MED NY VARIABEL ***********************************")
df_areal_innbyggerantall.show(50,False)

1.5 Finne maks og min

In [ ]:
# Skrive kode for å finne maks og min av befokningstetthet
from pyspark.sql.functions import mean, min, max
result = df_areal_innbyggerantall.select([min("Befolkningstetthet"), max("Befolkningstetthet")])
print("*********************************** MINIMUM OG MAKSIMUM AV VARIABELEN BEFOLKNINGSTETTHET ***********************************")
result.show()
